In [1]:
import io
import os
import re
import sys
import csv
import gzip
import time
import json
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
from scipy.stats import pearsonr
# import sqlite3
# from PIL import Image
import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
# from collections import Counter
# import seaborn as sns
# import matplotlib.colors as mcolors
# from matplotlib.ticker import FuncFormatter
import itertools
import collections
import pyreadr
import psa
import pywfa
from IPython.display import display_html
# import patchworklib as pw
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

# Genuine logics changed from the main script were marked by string **X** in the original R files.
# Simplicity changes were marked by string **Y**

<Figure size 100x100 with 0 Axes>

In [2]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0482.bam"

In [26]:
# def process_db_genfile(gene, 
#                             ipd_gen_file_dir, 
#                             hla_gene_information, 
#                             hlatypes,
#                             phased_vcf, 
#                             reference_allele_ary, 
#                             strict_snp_filter = True,
#                             read_from_QUILT = False, 
#                             subset_vcf_samples = None,
#                             sample_linker = None):
    
strand = hla_gene_information[hla_gene_information['Name'] == ('HLA-' + gene)]['Strand'].iloc[0]
#     reference_allele = reference_allele_ary[np.char.find(reference_allele_ary, gene) != -1][0]
nucleotides = ['A', 'T', 'C', 'G']

ipd_gen_file = ipd_gen_file_dir + gene + '_gen.txt'
with open(ipd_gen_file, "r") as file:
    lines = file.readlines()
gDNA_idx = []
names = []
i = 0
while len(gDNA_idx) < 2:
    l = lines[i]
    if 'gDNA' in l:
        gDNA_idx.append(i)
    elif l.lstrip(' ').split(' ')[0].startswith(gene + '*'):
        name = l.lstrip(' ').split(' ')[0]
        names.append(name)
    i += 1

first_base = int(lines[gDNA_idx[0]].split(' ')[-1].split('\n')[0])
n_alleles = gDNA_idx[1] - gDNA_idx[0] - 3

alleles_dict = {k:'' for k in names}
for i, s in enumerate(lines):
    r = s.lstrip(' ')
    if r.startswith(gene):
        r = r.rstrip(' \n')
        name = r.split(' ')[0]
        sequence = r.split(' ')[1:]
        sequence = ''.join(sequence)
        alleles_dict[name] = alleles_dict[name] + sequence

db = pd.DataFrame({key: list(value) for key, value in alleles_dict.items()}).T
db = db.drop(columns=db.columns[db.eq('|').all()])
db.columns = range(db.shape[1])

db = db.apply(lambda c: c.replace('-', c[0]) ,axis = 0)

if strand != 1:
    db.replace({'A': 't', 'C': 'g', 'G': 'c', 'T': 'a'}, inplace=True)
    db.replace({'a': 'A', 'c': 'C', 'g': 'G', 't': 'T'}, inplace=True)
    db = db.iloc[:, ::-1]

db

# #     variants_df = []
# #     for i in range(len(db)):
# #         variants_df.append(extract_snps_from_row(db.loc[reference_allele,:], db.iloc[i,], allele_pos))
# #     vcf = pd.concat(variants_df).drop_duplicates()
# #     vcf = vcf[(vcf['ref'].isin(nucleotides)) & (vcf['alt'].isin(nucleotides))].sort_values(by = 'pos').reset_index(drop = True)

# #     allele_pos = np.array(allele_pos)
# dbcopy = pd.DataFrame()
# #     aligned_df = pd.DataFrame()
# variants_to_remove = []

# for i in range(vcf.shape[0]):
#     pos, a1, a2 = vcf.iloc[i, :3]
#     index = np.where((allele_pos == pos))[0]

#     if len(index) == 1:
#         index = index[0]
#         dbcopy[dbcopy.shape[1]] = db[index]        
# #             aligned_df[aligned_df.shape[1]] = db[index].apply(recode_variant, args = (a2))
#     else:
#         variants_to_remove.append(i)

# #     vcf = vcf[~vcf.index.isin(variants_to_remove)].reset_index(drop = True) 
# # This is a remedy when extract_snps_from_row fails to detect insertions. 
# # For DRB1 we get 2442 variants instead of 2452, fairly good I'd say
# dbfull = db
# db = dbcopy

# #     sample = pd.DataFrame({
# #         "ID_1": aligned_df.index,
# #         "ID_2": aligned_df.index,
# #         "missing": 0
# #     })
# #     vcf['id'] = 'chr6:' + vcf['pos'].astype(str)
# #     haps = aligned_df.T
# #     cond = (haps == -1).mean(axis=1) < 0.1
# #     haps = haps[cond].reset_index(drop = True)
# #     vcf = vcf[cond].reset_index(drop = True)

# #     start = vcf.loc[0, 'pos']
# #     end =vcf.loc[len(vcf) - 1, 'pos']

# #     cond = (allele_pos >= start) & (allele_pos <= end)
# #     allele_pos = allele_pos[cond]
# #     dbfull = dbfull.iloc[:, np.where(cond == True)[0]]
# #     dbfull.columns = range(dbfull.shape[1])

# #     # Imputing database alleles
# #     for i, a in enumerate(haps.columns):
# #         haps_undetermined = haps[a] == -1

# #         allele_col = haps[~haps_undetermined][a]
# #         subseted_db =  haps[~haps_undetermined] 
# #         pairwise_distance = subseted_db.sub(subseted_db[a], axis=0)
# #         counts = pairwise_distance.abs().sum().sort_values().iloc[1:]
# #         closest_allele_idx = 0
# #         substituted_nucleotides = np.array([np.nan]*np.sum(haps_undetermined))
# #         substituted_indices = haps_undetermined[haps_undetermined == True].index

# #         while np.isnan(substituted_nucleotides).any() > 0:
# #             closest_allele = counts.index[closest_allele_idx]

# #             target_haplotype = haps.loc[substituted_indices, closest_allele]
# #             substitutable = target_haplotype[target_haplotype.values != -1]
# #             substituted_nucleotides[np.where(np.in1d(substituted_indices, substitutable.index))[0]] = substitutable.values
# #             closest_allele_idx += 1

# #         haps.loc[substituted_indices, a] = substituted_nucleotides
# #         # haps has dim n_variants * n_alleles

,18517,18516,18515,18514,18513,18512,18511,18510,18509,18508,...,9,8,7,6,5,4,3,2,1,0
DRB1*01:01:01:01,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:02,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:03,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:04,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*01:01:01:05,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DRB1*16:02:01:06,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:02:10,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:05:01,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
DRB1*16:09:01,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [21]:
regstart = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Start'].values[0]
regend = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['End'].values[0]
strand = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Strand'].values[0]

reads_apart_max = 1000

command = f"samtools view {bam} chr6:{regstart-reads_apart_max}-{regend+reads_apart_max}"
reads = subprocess.run(command, shell = True, capture_output = True, text = True).stdout[:-1].split('\n')

reads = [i.split('\t') for i in reads if '##' not in i]
reads = pd.DataFrame(reads).iloc[:,:11]
reads.columns = [
    "ID", "flag", "chr", "pos", "map_quality", "CIGAR", "chr_alt", "pos_alt", "insert_size", "sequence", "base_quality"
]

reads['pos'] = reads['pos'].astype(int)
reads['pos_alt'] = reads['pos_alt'].astype(int)

reads = reads[
    (reads['chr'] == 'chr6') &
    (reads['chr_alt'].isin(['chr6', '='])) &
    (reads['pos_alt'] + reads['sequence'].str.len() >= np.ones(len(reads))*(regstart - reads_apart_max)) & 
    (reads['pos_alt'] <= np.ones(len(reads))*(regend + reads_apart_max))
]

id_counts = reads['ID'].value_counts()
valid_ids_ary = id_counts[id_counts == 2].index.tolist()
reads = reads[reads['ID'].isin(valid_ids_ary)].sort_values(by = 'ID').reset_index(drop = True)

reads

,ID,flag,chr,pos,map_quality,CIGAR,chr_alt,pos_alt,insert_size,sequence,base_quality
0,A00711:748:HHFNVDSX7:4:1164:15157:28463,163,chr6,32580978,60,151M,=,32581049,222,GCCATTTCGGGAGAAAAAAAGGATTTCAAATCACACTGAACAGTTA...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
1,A00711:748:HHFNVDSX7:4:1164:15157:28463,83,chr6,32581049,60,151M,=,32580978,-222,AAATATTACAGCCTTGATGTAAGGCAAGAGTTCAACATCTGATCCA...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFF...
2,A00711:748:HHFNVDSX7:4:1501:14696:8249,83,chr6,32589048,60,139M12S,=,32588932,-255,AAAGAAAGAGCACTAAGTATAACTTCTGTCAGACAACCTACATACA...,"FFF:FFFFFFFFFFFFFFFFF:FFFFFFF:FFFFFFFF,FF:FFFF..."
3,A00711:748:HHFNVDSX7:4:1501:14696:8249,163,chr6,32588932,60,151M,=,32589048,255,GAGGATGCCTTAAGTTCTTTAGGCACCAAAGAATACCTCATAAATG...,FFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
4,A00711:748:HHFNVDSX7:4:1619:32280:8046,83,chr6,32583021,60,151M,=,32582884,-288,CATACAAACCACAAACTGGAGCAAATGCTGAATCAAATATCAATAA...,"FFFFFFFFF:FFFFFFFF,FFFF,FFFFFFFFFFFFFFFFFFFFFF..."
...,...,...,...,...,...,...,...,...,...,...,...
79,A00711:753:HHCG3DSX7:4:2570:5222:23093,147,chr6,32590480,40,151M,=,32590322,-309,TCTTAGTAAAAAGAGTTTTTTCAAGTGTATCTCCTGATGCTTCCCC...,"FFFF,FFFFFFF::FF:FFFFFFFFFF:FFFF:FFFFF:F:FFFFF..."
80,A00711:753:HHCG3DSX7:4:2635:21630:12649,99,chr6,32590175,51,151M,=,32590246,218,AGATGGAGGGAACATGGTTGAAAATCAGGACACCTTTAATCTGGTC...,FFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
81,A00711:753:HHCG3DSX7:4:2635:21630:12649,147,chr6,32590246,40,79M3I36M1I32M,=,32590175,-218,TAGATTTGGGAAAATTACTTCATGTCTCACAGTTGATATGAAGGCA...,FFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFF...
82,A00711:753:HHCG3DSX7:4:2640:3405:19069,163,chr6,32578686,60,37M10I104M,=,32578872,337,GTGAAAATCATTATTCTGTTAGGAAATAATCTTCCCTTTTTCTTTT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFF:FFFFFFF...


In [17]:
from pywfa import WavefrontAligner

pattern = "*CTTTACTCGCGCGTTGGAGAAATAAAATAGT"
text =      "*CTTTACTCGCGCGTTTGGAGAAATAAAATAGT"
a = WavefrontAligner(pattern)
score = a.wavefront_align(text)
score

-8

In [12]:
import pywfa

pattern = "AAAAACCTTTTTAAAAAA"
text =       "GGCCAAAAACCAAAAAA"
a = WavefrontAligner(pattern)

res = a(text, clip_cigar=False)
print(pywfa.cigartuples_to_str(res.cigartuples))

print(a.wavefront_align(text))

4I7M5D6M
-30


In [56]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')

ref_samples = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/oneKG.samples', sep = ' ')
hlatypes = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/20181129_HLA_types_full_1000_Genomes_Project_panel.txt', sep = '\t')
ref_samples_removed = ref_samples[~ref_samples['SAMPLE'].isin(hlatypes['Sample ID'].tolist())]
samples_to_remove = ref_samples_removed['SAMPLE'].tolist()
hlatypes = hlatypes[~hlatypes['Sample ID'].isin(samples_to_remove)].sort_values(by = 'Sample ID').reset_index(drop = True)

phased_vcf = "/well/band/users/rbx225/recyclable_files/ref_panels/oneKG/oneKG.chr6.vcf.gz"
reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))

# subset_samples = None

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information,
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = False, 
                            subset_vcf_samples = None,
                            sample_linker = None)

individual = 'NA12878'
ix = hlatypes.index[hlatypes['Sample ID'] == 'NA12878'][0]
display_indices = np.arange(10)

res = visualise_phase(gene, ix, hlatypes, return_dict)
compare_phase(display_indices, res)

# reference_exclude_samplelist_file = ''
# if reference_exclude_samplelist_file != '':
#     samples_to_remove = samples_to_remove + lcwgsus.read_tsv_as_lst(reference_exclude_samplelist_file)
# if samples_to_remove is not none, save to "hlauntyped.exclude.txt"

In [ ]:
# For GAMCC
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"

subset_vcf_samples = lcwgsus.read_tsv_as_lst('/well/band/users/rbx225/GAMCC/data/sample_tsvs/fv_gm_names.tsv')
subset_vcf_samples = ','.join(subset_vcf_samples)

sample_linker = pd.read_csv('/well/band/users/rbx225/GAMCC/' + SAMPLE_LINKER_FILE)
sample_linker = {k:v for k, v in zip(sample_linker['Sample_Name'], sample_linker['Chip_Name'])}

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information, 
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = True, 
                            subset_vcf_samples = subset_vcf_samples,
                            sample_linker = sample_linker)